## Methodische & Digitale Kompetenz (DIKO) - Modularbeit Eigenleistung - SS 2024 - Maximilian Fiebig
## Experiment Systemprompt

#### Installation von OpenAI-Bibliothek (https://pypi.org/project/openai/)

In [1]:
pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


#### Erstellung der Azure OpenAI-Instanz (https://learn.microsoft.com/de-de/azure/ai-services/openai/how-to/switching-endpoints#api-key)
- Der API KEY ist ähnlich wie ein Passwort
- Man speichert ihn in einer externen .env-Datei, damit niemand den Key im Code sehen kann
- Die API-Version ist die genaue Spezifikation/Anleitung, damit die Bibliothek genau weiß, wie das JSON formatiert werden muss. Sie ändert sich durch Updates, deswegen muss sie mit angegeben werden. Die Version 2024-02-01 ist die von MUCGPT genutze Version bei Azure. (https://learn.microsoft.com/en-gb/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release)

In [2]:
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv() #Laden der externen .env-Datei, in der API-Key und Azure Endpoint gespeichert sind.

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

Chat Completions API-Aufruf: https://platform.openai.com/docs/guides/text-generation/chat-completions-api

- Temperature: Im Prompting bestimmt die Temperatur die Gewichtung von Kreativität in den Antworten der KI. Sie kann auch die Konsistenz und Vorhersehbarkeit der KI-Antworten beeinflussen. Eine niedrigere Temperatur führt dazu, dass die KI eher Wörter wählt, die wahrscheinlicher sind, und die Antworten beim gleichen Prompt sich ähnlich sind. Näheres dazu wird in der MUCGPT_temperature.ipynb Datei erläutert.

In [3]:
response = client.chat.completions.create(
  model="chat", # 'Chat' ist das zugehörige Deployment bei Azure, dahinter liegt gpt-3.5-turbo.
  messages=[
    {"role": "system", "content": "Du bist ein Experte für Baustoffkunde."}, # Anweisung an das Sprachmodell, optional
    {"role": "user", "content": "Was sind die Bestandteile von Beton?"} # Frage von Nutzer an das Sprachmodell
  ], 
  temperature=1 # Steuerung der Kreativität
)

In [4]:
print(response.choices[0].message.content) # Ausgabe der Antwort des Chatmodells

Die Bestandteile von Beton sind Zement, Wasser, Gesteinskörnung (Sand, Kies oder Schotter) und gegebenenfalls Zusatzstoffe wie beispielsweise Chemikalien zur Verbesserung der Verarbeitbarkeit oder der Festigkeit. Zusammen ergeben diese Bestandteile eine homogene Masse, die nach dem Aushärten eine hohe Festigkeit aufweist und für den Bau von Gebäuden, Straßen, Brücken und vielen anderen Konstruktionen genutzt werden kann.


#### Importieren der .txt-Datei mit Fragen

In [5]:
with open("Fragen.txt") as datei:
    fragen = datei.readlines() # In der txt.-Datei sollen alle Zeilen gelesen werden.

print(fragen) # Ausgabe der Fragen

['Wie viel wiegt 1m³ Beton?\n', 'Gibt es klimaneutrale Baustoffe?\n', 'Welche Eigenschaften hat Asphalt?\n', 'Wie wird ein Ziegelstein hergestellt?\n', 'Kann man eine Straße aus Beton bauen?\n', 'Aus welchem Baustoff bestehen Brücken?\n', 'Wie sieht die Rezyklierbarkeit von Beton aus?\n', 'Ist Stahl ein guter Baustoff für U-Bahngleise?\n', 'Ist es möglich, ein Haus aus dem Baustoff Holz zu bauen?\n', 'Welches Material wird für den Einsatz in Hochbauten eingesetzt?']


In [6]:
fragen = [line.rstrip() for line in fragen] # .rstrip entfernt Zeilenumbürche (/n), https://statistikguru.de/python/python-zeilenweise-lesen.html
print(fragen)

['Wie viel wiegt 1m³ Beton?', 'Gibt es klimaneutrale Baustoffe?', 'Welche Eigenschaften hat Asphalt?', 'Wie wird ein Ziegelstein hergestellt?', 'Kann man eine Straße aus Beton bauen?', 'Aus welchem Baustoff bestehen Brücken?', 'Wie sieht die Rezyklierbarkeit von Beton aus?', 'Ist Stahl ein guter Baustoff für U-Bahngleise?', 'Ist es möglich, ein Haus aus dem Baustoff Holz zu bauen?', 'Welches Material wird für den Einsatz in Hochbauten eingesetzt?']


#### Erstellen von Funktion und Liste zum auslesen der Fragen und Antworten

In [7]:
def frage_gpt(text, prompt): # Definieren der Funktion frage_gpt und Übergabe der Eingabewerte. 'text' fungiert als Platzhalter, Temperature wird auf 0 festgelegt.
    response = client.chat.completions.create(
        model="chat",
        messages=[
            {"role": "system", "content": prompt}, # Die Anweisung an das Sprachmodell erfolgt weiter unten, beim festlegen der Prompts.
            {"role": "user", "content": text}
        ], 
        temperature=0.7
    )
    return response.choices[0].message.content

In [8]:
import os
os.remove('Ausgabe_Prompt.txt') #Öffnen der Datei Ausgabe_Prompt.txt, um die Fragen und Antworten in dieser Datei auszugeben.

antworten = [] # Erstellen einer Liste für die Ausgabe der Antworten.

file = open('Ausgabe_Prompt.txt', 'a')
for frage in fragen: # Schleife, um alle Fragen aus der Liste abzuarbeiten und die entsprechende Antwort des Chatmodells mit abzufragen.
    antwort_ohne_prompt = frage_gpt(frage, prompt="") # Festlegen der verschiedenen Rollen/Prompts des Chatmodells
    antwort_experte = frage_gpt(frage, prompt="Du bist ein Experte für Bauingenieurwesen. Antworte kurz & prägnant auf Fragen.")
    antwort_dichter = frage_gpt(frage, prompt="Du bist ein Dichter aus dem 16 Jahrhundert, der sich gerne mit Natur beschäftigt. Reime deine Antworten.")
    antwort_satire = frage_gpt(frage, prompt="Du bist ein Satire-Bot für Bauwesen. Gib nur skurille Falschantworten.")
    output_text = "Frage: {}\nAntwort Ohne Prompt: {}\nAntwort Experte: {}\nAntwort Dichter: {}\nAntwort Satire: {}\n\n".format(frage, antwort_ohne_prompt, antwort_experte, antwort_dichter, antwort_satire) # Formatierung der Antwortausgabe.
    file.write(output_text) # Übergabe der Antwort an die .txt-Ausgabedatei.
    print(output_text)

file.close()

Frage: Wie viel wiegt 1m³ Beton?
Antwort Ohne Prompt: Das Gewicht von 1m³ Beton hängt von der Art des Betons ab. Ein normaler Beton hat eine Dichte von etwa 2.400 kg/m³, während ein Leichtbeton eine Dichte von etwa 1.600 kg/m³ hat. Das bedeutet, dass 1m³ normaler Beton etwa 2.400 kg wiegt, während 1m³ Leichtbeton etwa 1.600 kg wiegt.
Antwort Experte: Das Gewicht von 1m³ Beton variiert je nach Art des Betons, aber im Durchschnitt wiegt 1m³ Beton etwa 2.400 kg bis 2.500 kg (2,4 bis 2,5 Tonnen).
Antwort Dichter: Ein Kubikmeter Beton wiegt schwer,
etwa 2,4 Tonnen, das ist nicht mehr,
so viel Masse auf kleinem Raum,
im Garten ist das kein Traum.
Antwort Satire: Das hängt davon ab, ob der Beton mit Ziegelsteinen oder Marshmallows hergestellt wurde. In beiden Fällen ist das Gewicht jedoch garantiert lecker.


Frage: Gibt es klimaneutrale Baustoffe?
Antwort Ohne Prompt: Ja, es gibt klimaneutrale Baustoffe, die dazu beitragen können, den CO2-Fußabdruck von Gebäuden zu reduzieren. Einige Beispie

#### Erkenntnis: Grundsätzlich ist ein Prompt eine Eingabeaufforderung, die verwendet wird, um einen Computer oder eine KI-Plattform zu instruieren, einen bestimmten Text zu generieren oder eine besondere Aufgabe zu erledigen (https://bm-experts.de/definitionenfaq/definitionen/prompt-was-ist-das-und-wie-kann-er-eingesetzt-werden/). Nachdem die drei verschiedenen Prompts im Code eingegeben wurden, kann man in der Ausgabe die Antworten betrachten. Hierbei lässt sich klar erkennen, dass das Chatmodell nach den eingegebenen Vorgaben arbeitet und die Antworten mehr oder weniger so ausgibt wie gewünscht. Die Konsistenz der Aussagen ist hierbei noch verbesserungswürdig, vorallem beim Prompt: Dichter (Reimen der Antworten) kommt die KI an ihre Grenzen.

